In [1]:
import math
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy.stats import mode
import csv
import matplotlib.dates
from datetime import *
from sklearn.preprocessing import *
from sklearn import ensemble
import xgboost as xgb
from sklearn import metrics
from xgboost.sklearn import XGBClassifier  
from sklearn.model_selection import GridSearchCV,cross_val_score  
import matplotlib.pylab as plt  

from sklearn.preprocessing import *
import xgboost as xgb
from sklearn import metrics
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.externals import joblib

In [19]:
#首先导入训练集和测试集数据
jdata_train_df = pd.read_csv('chusai_jdata_train_df.csv')
jdata_test_df = pd.read_csv('chusai_jdata_test_df.csv')

fea = [
        
    'age', 'sex', 'user_lv_cd', 'year', 'month',
     'last_buy_cate101_deltaDay', 'last_buy_cate30_deltaDay',
     'last_buy_cate1_deltaDay', 'last_buy_cate71_deltaDay',
     'last_buy_cate46_deltaDay', 'last_buy_cate83_deltaDay',
     'last_focus_cate101_number', 'last_focus_cate30_number',
     'last_focus_cate1_number', 'last_focus_cate71_number',
     'last_focus_cate46_number', 'last_focus_cate83_number',
     'last_click_cate101_deltaDay', 'last_click_cate30_deltaDay',
     'last_click_cate1_deltaDay', 'last_click_cate71_deltaDay',
     'last_click_cate46_deltaDay', 'last_click_cate83_deltaDay',
     'last_buy_cate101_number', 'last_buy_cate30_number', 'last_buy_cate1_number',
     'last_buy_cate71_number', 'last_buy_cate46_number', 'last_buy_cate83_number',
     'user_targetCate_monthlyNumber', 'user_relatedCate_monthlyNumber',
     'user_30Cate_monthlyNumber', 'user_101Cate_monthlyNumber',
     'user_targetCate_monthlyOrderNumber', 'user_relatedCate_monthlyOrderNumber',
     'user_targetCate_monthlyClickNumber', 'user_relatedCate_monthlyClickNumber',  
     'user_targetCate_monthlyBuyRate', 'user_targetCate_monthlyClickRate',
     'user_comment_count', 'user_praiseComment_count', 'user_badComment_count',
     'user_praiseComment_rate', 'user_badComment_rate',
#      'user_cate101_orderDayMax', 'user_cate101_orderDayMean',
#      'user_cate101_orderDayMin', 'user_cate30_orderDayMax',
#      'user_cate30_orderDayMean', 'user_cate30_orderDayMin',
#      'user_targetCate_orderDayMax', 'user_targetCate_orderDayMean',
#      'user_targetCate_orderDayMin', 'user_relatedCate_orderDayMax',
#      'user_relatedCate_orderDayMean', 'user_relatedCate_orderDayMin',
    
      ]

print(len(jdata_train_df.columns))
print(len(jdata_test_df.columns))

train_dataset1_x = jdata_train_df[fea][(jdata_train_df.month <= 3) | (jdata_train_df.month >= 11)]
train_dataset1_y = jdata_train_df['is_order'][(jdata_train_df.month <= 3) | (jdata_train_df.month >= 11)]
test_dataset1_x = jdata_train_df[fea][jdata_train_df.month == 4]
test_dataset1_y = jdata_train_df['is_order'][jdata_train_df.month == 4]

train_dataset_x = jdata_train_df[fea][(jdata_train_df.month <= 4) | (jdata_train_df.month >= 11)]
train_dataset_y = jdata_train_df['is_order'][(jdata_train_df.month <= 4) | (jdata_train_df.month >= 11)]




63
61


In [20]:
#开始进行xgb模型训练
xgb_train_dataset1 = xgb.DMatrix(train_dataset1_x,label = train_dataset1_y)
xgb_test_dataset1 = xgb.DMatrix(test_dataset1_x, label = test_dataset1_y)

cv_params={'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric':'logloss',
        'gamma':0.7,
        'min_child_weight':11,
        'max_depth':4,
        'lambda':9,
        'subsample':0.9,
        'colsample_bytree':0.7,
        'colsample_bylevel':0.7,
        'eta': 0.1,
        'tree_method':'exact',
        'seed':0,
        'nthread':12
        }

offline_watchlist = [(xgb_train_dataset1,'train'), (xgb_test_dataset1, 'val')]
offline_model = xgb.train(cv_params, xgb_train_dataset1, num_boost_round=510, evals=offline_watchlist)

# params={'booster':'gbtree',
#         'objective': 'binary:logistic',
#         'eval_metric':'logloss',
#         'gamma':0.1,                               #6
#         'min_child_weight':5,                      #2
#         'max_depth':3,                             #1
#         'lambda':11,                               #5
#         'subsample':0.85,                          #3
#         'colsample_bytree':0.65,                    #4
#         'colsample_bylevel':0.7,
#         'scale_pos_weight':1,
#         'eta': 0.1,
#         'tree_method':'exact',
#         'seed':0,
#         'nthread':12
#         }

# offline_watchlist = [(xgb_train_dataset1,'train'), (xgb_test_dataset1, 'val')]
# offline_model = xgb.train(params, xgb_train_dataset1, num_boost_round=200, evals=offline_watchlist)

#predict test set
test_dataset1_x['predicted_is_order'] = offline_model.predict(xgb_test_dataset1)


[0]	train-logloss:0.672922	val-logloss:0.693144
[1]	train-logloss:0.648857	val-logloss:0.687568
[2]	train-logloss:0.629031	val-logloss:0.683692
[3]	train-logloss:0.613977	val-logloss:0.682056
[4]	train-logloss:0.599788	val-logloss:0.680453
[5]	train-logloss:0.587512	val-logloss:0.677886
[6]	train-logloss:0.576948	val-logloss:0.673945
[7]	train-logloss:0.566125	val-logloss:0.671067
[8]	train-logloss:0.557215	val-logloss:0.67033
[9]	train-logloss:0.549191	val-logloss:0.669551
[10]	train-logloss:0.542315	val-logloss:0.66808
[11]	train-logloss:0.537763	val-logloss:0.669349
[12]	train-logloss:0.533076	val-logloss:0.669527
[13]	train-logloss:0.52916	val-logloss:0.66901
[14]	train-logloss:0.524664	val-logloss:0.666852
[15]	train-logloss:0.521617	val-logloss:0.665438
[16]	train-logloss:0.519191	val-logloss:0.664348
[17]	train-logloss:0.51663	val-logloss:0.663368
[18]	train-logloss:0.514578	val-logloss:0.66466
[19]	train-logloss:0.511989	val-logloss:0.663468
[20]	train-logloss:0.509784	val-logl

[167]	train-logloss:0.465202	val-logloss:0.591354
[168]	train-logloss:0.465135	val-logloss:0.590584
[169]	train-logloss:0.46509	val-logloss:0.590474
[170]	train-logloss:0.465008	val-logloss:0.591275
[171]	train-logloss:0.464891	val-logloss:0.59073
[172]	train-logloss:0.464861	val-logloss:0.590668
[173]	train-logloss:0.464753	val-logloss:0.590041
[174]	train-logloss:0.464676	val-logloss:0.590082
[175]	train-logloss:0.464631	val-logloss:0.589945
[176]	train-logloss:0.464597	val-logloss:0.589848
[177]	train-logloss:0.464569	val-logloss:0.589764
[178]	train-logloss:0.464492	val-logloss:0.590022
[179]	train-logloss:0.464444	val-logloss:0.590054
[180]	train-logloss:0.464317	val-logloss:0.589864
[181]	train-logloss:0.464265	val-logloss:0.589903
[182]	train-logloss:0.464242	val-logloss:0.589954
[183]	train-logloss:0.464152	val-logloss:0.589338
[184]	train-logloss:0.464074	val-logloss:0.58932
[185]	train-logloss:0.464025	val-logloss:0.589201
[186]	train-logloss:0.463965	val-logloss:0.589237
[18

[332]	train-logloss:0.459393	val-logloss:0.587146
[333]	train-logloss:0.459381	val-logloss:0.587204
[334]	train-logloss:0.459368	val-logloss:0.587275
[335]	train-logloss:0.459335	val-logloss:0.587241
[336]	train-logloss:0.459293	val-logloss:0.587095
[337]	train-logloss:0.459276	val-logloss:0.587083
[338]	train-logloss:0.459257	val-logloss:0.587099
[339]	train-logloss:0.459242	val-logloss:0.587141
[340]	train-logloss:0.459226	val-logloss:0.587129
[341]	train-logloss:0.459217	val-logloss:0.587278
[342]	train-logloss:0.459196	val-logloss:0.587292
[343]	train-logloss:0.45918	val-logloss:0.587281
[344]	train-logloss:0.459162	val-logloss:0.587166
[345]	train-logloss:0.459155	val-logloss:0.587247
[346]	train-logloss:0.459135	val-logloss:0.587311
[347]	train-logloss:0.459127	val-logloss:0.587288
[348]	train-logloss:0.459097	val-logloss:0.587161
[349]	train-logloss:0.459082	val-logloss:0.587135
[350]	train-logloss:0.459046	val-logloss:0.587223
[351]	train-logloss:0.459039	val-logloss:0.587234
[

[497]	train-logloss:0.456504	val-logloss:0.585318
[498]	train-logloss:0.456483	val-logloss:0.585275
[499]	train-logloss:0.456464	val-logloss:0.585083
[500]	train-logloss:0.456449	val-logloss:0.585081
[501]	train-logloss:0.456402	val-logloss:0.584638
[502]	train-logloss:0.456391	val-logloss:0.584626
[503]	train-logloss:0.456383	val-logloss:0.584643
[504]	train-logloss:0.45637	val-logloss:0.584617
[505]	train-logloss:0.456336	val-logloss:0.584564
[506]	train-logloss:0.456305	val-logloss:0.584551
[507]	train-logloss:0.456301	val-logloss:0.584583
[508]	train-logloss:0.456278	val-logloss:0.584589
[509]	train-logloss:0.456243	val-logloss:0.584445


In [21]:
#输出线下训练的xgb模型的重要性
feat_imp = pd.Series(offline_model.get_fscore()).sort_values(ascending=False)
print(feat_imp.head(100))


user_targetCate_monthlyOrderNumber     625
last_buy_cate101_deltaDay              538
month                                  401
last_buy_cate30_deltaDay               365
user_targetCate_monthlyClickNumber     341
last_click_cate101_deltaDay            340
user_targetCate_monthlyNumber          311
user_101Cate_monthlyNumber             270
user_targetCate_monthlyClickRate       262
last_click_cate30_deltaDay             255
last_buy_cate101_number                246
user_30Cate_monthlyNumber              197
last_buy_cate30_number                 188
user_relatedCate_monthlyClickNumber    180
user_targetCate_monthlyBuyRate         177
year                                   173
last_click_cate71_deltaDay             159
user_lv_cd                             139
last_click_cate46_deltaDay             134
age                                    130
last_buy_cate71_deltaDay               120
last_buy_cate46_deltaDay               117
user_relatedCate_monthlyNumber         116
last_click_

In [22]:
test_dataset1_x['user_id'] = jdata_train_df['user_id'][jdata_train_df.month == 4]
test_dataset1_x = test_dataset1_x.sort_index(by='predicted_is_order', ascending=False)
order_jdata_test_dataset1_x = test_dataset1_x[['user_id', 'predicted_is_order']][:50000]
print("线下验证集预测结果：\n", order_jdata_test_dataset1_x[['user_id','predicted_is_order']].head())


线下验证集预测结果：
        user_id  predicted_is_order
35584    35585            0.984826
80494    80495            0.982752
67216    67217            0.980887
10892    10893            0.978830
7564      7565            0.978439


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  


In [23]:
#定义计算s1分数的函数
def s1Score(predict_user_list, real_user_set):
    i = 1
    oi = 0
    wi = 0
    for user_id in predict_user_list.values:
        if user_id in real_user_set:
            oi = oi + (1 / (1 + math.log(i)))
        wi = wi + (1 / (1 + math.log(i)))
        i = i + 1
    s1 = oi / wi
    print(i, s1)
    return s1

predict_user_list = order_jdata_test_dataset1_x['user_id']
real_user_set = set(jdata_train_df['user_id'][(jdata_train_df.is_order == 1) & (jdata_train_df.month == 4)])
s1 = s1Score(predict_user_list, real_user_set)


50001 0.6873589435569524


In [25]:
#进行正式模型训练
xgb_train = xgb.DMatrix(train_dataset_x,label=train_dataset_y)
xgb_test = xgb.DMatrix(jdata_test_df[fea])

cv_params={'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric':'logloss',
        'gamma':0.7,
        'min_child_weight':11,
        'max_depth':4,
        'lambda':9,
        'subsample':0.9,
        'colsample_bytree':0.7,
        'colsample_bylevel':0.7,
        'eta': 0.1,
        'tree_method':'exact',
        'seed':0,
        'nthread':12
        }

watchlist = [(xgb_train,'train')]
model = xgb.train(cv_params, xgb_train, num_boost_round=510, evals=watchlist)

# params={'booster':'gbtree',
#         'objective': 'binary:logistic',
#         'eval_metric':'logloss',
#         'gamma':0.1,                               #6
#         'min_child_weight':5,                      #2
#         'max_depth':3,                             #1
#         'lambda':11,                               #5
#         'subsample':0.85,                          #3
#         'colsample_bytree':0.65,                    #4
#         'colsample_bylevel':0.7,
#         'scale_pos_weight':1,
#         'eta': 0.1,
#         'tree_method':'exact',
#         'seed':0,
#         'nthread':12
#         }

# watchlist = [(xgb_train,'train')]
# model = xgb.train(params, xgb_train, num_boost_round=200, evals=watchlist)



[0]	train-logloss:0.667747
[1]	train-logloss:0.647037
[2]	train-logloss:0.633307
[3]	train-logloss:0.61844
[4]	train-logloss:0.607827
[5]	train-logloss:0.597686
[6]	train-logloss:0.588293
[7]	train-logloss:0.57992
[8]	train-logloss:0.574099
[9]	train-logloss:0.567874
[10]	train-logloss:0.561681
[11]	train-logloss:0.557128
[12]	train-logloss:0.553617
[13]	train-logloss:0.548469
[14]	train-logloss:0.545914
[15]	train-logloss:0.542211
[16]	train-logloss:0.538353
[17]	train-logloss:0.535066
[18]	train-logloss:0.532494
[19]	train-logloss:0.529708
[20]	train-logloss:0.526072
[21]	train-logloss:0.523557
[22]	train-logloss:0.52049
[23]	train-logloss:0.519013
[24]	train-logloss:0.516951
[25]	train-logloss:0.514636
[26]	train-logloss:0.512881
[27]	train-logloss:0.510314
[28]	train-logloss:0.508963
[29]	train-logloss:0.50775
[30]	train-logloss:0.506665
[31]	train-logloss:0.505787
[32]	train-logloss:0.504313
[33]	train-logloss:0.503094
[34]	train-logloss:0.502601
[35]	train-logloss:0.500973
[36]	t

[288]	train-logloss:0.456814
[289]	train-logloss:0.456799
[290]	train-logloss:0.456788
[291]	train-logloss:0.456781
[292]	train-logloss:0.456756
[293]	train-logloss:0.45675
[294]	train-logloss:0.456224
[295]	train-logloss:0.456215
[296]	train-logloss:0.456184
[297]	train-logloss:0.456173
[298]	train-logloss:0.456163
[299]	train-logloss:0.456136
[300]	train-logloss:0.456091
[301]	train-logloss:0.456052
[302]	train-logloss:0.456039
[303]	train-logloss:0.456014
[304]	train-logloss:0.455845
[305]	train-logloss:0.455813
[306]	train-logloss:0.455799
[307]	train-logloss:0.45577
[308]	train-logloss:0.455757
[309]	train-logloss:0.45573
[310]	train-logloss:0.455693
[311]	train-logloss:0.45569
[312]	train-logloss:0.455683
[313]	train-logloss:0.45556
[314]	train-logloss:0.455243
[315]	train-logloss:0.455226
[316]	train-logloss:0.45519
[317]	train-logloss:0.455124
[318]	train-logloss:0.455087
[319]	train-logloss:0.455077
[320]	train-logloss:0.455071
[321]	train-logloss:0.455045
[322]	train-logloss:

In [26]:
#predict test set
jdata_test_df['predicted_is_order'] = model.predict(xgb_test)
jdata_test_df = jdata_test_df.sort_index(by='predicted_is_order', ascending=False)
order_jdata_test_df = jdata_test_df[['user_id', 'predicted_is_order']][:50000]
print(order_jdata_test_df.head(10))


       user_id  predicted_is_order
64724    64725            0.978145
892        893            0.976353
242        243            0.975338
15897    15898            0.975307
24981    24982            0.975242
54543    54544            0.974824
84351    84352            0.974539
19372    19373            0.974210
65048    65049            0.973724
13032    13033            0.973521


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
from datetime import  datetime
order_jdata_test_df['pred_date'] = datetime(2017, 5, 15)

print(order_jdata_test_df.head(10))


       user_id  predicted_is_order  pred_date
64724    64725            0.978145 2017-05-15
892        893            0.976353 2017-05-15
242        243            0.975338 2017-05-15
15897    15898            0.975307 2017-05-15
24981    24982            0.975242 2017-05-15
54543    54544            0.974824 2017-05-15
84351    84352            0.974539 2017-05-15
19372    19373            0.974210 2017-05-15
65048    65049            0.973724 2017-05-15
13032    13033            0.973521 2017-05-15


In [28]:
# 导出预测结果
def exportResult(df, fileName):
    df.to_csv('../../data/chusai/submit/%s.csv' % fileName, header=True, index=False)
    
exportResult(order_jdata_test_df[['user_id', 'pred_date']], 'jdata_a_5_25_1')
